# Maheen Shoukat
## i212719 - BAI-6J
### NLP A2 TASK2

In [1]:
# Load the Poetry Corpus
# Tokenize the corpus in order to split it into a list of words
# Generate n-gram models
# For each of the stanzas
    # For each verse
        # Generate a random number in the range [7...10]
        # Select first word
        # Select subsequent words until end of verse
        #[bonus]If not the first verse, try to rhyme the last word with the last word of the previous verse
        # Print verse
# Print empty line after stanza

In [2]:
# txt processing -> spacy
# n-grams 
# to generate : task -> 4 stanzas, each verse 7-10 words

In [3]:
# subsequent words -> cfd -> likelihood of next word
# Rhyming verses -> your dictionary for rhyming

# 1) unigram model
# 2) bigram model -> Select the first word of each line randomly from starting words in the vocabulary
#     and then use the bigram model to generate the next word until the verse is complete.
# 3) trigram model -> Select the first word of each line randomly from starting words in the vocabulary
#     and then use the trigram model to generate the next word until the verse is complete.
# 4) compare bigram and trigram performance. -> make 4 stanzas rhyme

# 5) backward bigram model -> model the generation of a sentence from right to left.
# 6) compare backward bigram model with other ngram models

# 7) bidirectional bigram model -> merge fwd and backward

In [4]:
import numpy as np
import csv
import spacy
import string
from collections import Counter
import random
from bs4 import BeautifulSoup
import re

C:\Users\DELL\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load the Poetry Corpus

In [5]:
# with open('corpus.txt', 'rt', encoding="utf-8") as f:
#     reader = csv.reader(f)
#     poetry = list(reader)
# f.close()
with open(r'D:\Semester VI\NLP\i212719_Scrapy\urdupoems\poet_poems.csv', 'rt', encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader)
#     poetry = list(reader)
    poetry = [row[1] for row in reader]

f.close()

In [6]:
len(poetry)

27104

In [7]:
poetry[:10]

['آپ  کے  بعد  ہر  گھڑی  ہم  نے  آپ  کے  ساتھ  ہی  گزاری  ہے',
 'آپ  کے  بعد  ہر  گھڑی  ہم  نے  آپ  کے  ساتھ  ہی  گزاری  ہے',
 'آئنہ  دیکھ  کر  تسلی  ہوئی  ہم  کو  اس  گھر  میں  جانتا  ہے  کوئی',
 'آپ  کے  بعد  ہر  گھڑی  ہم  نے  آپ  کے  ساتھ  ہی  گزاری  ہے',
 'آئنہ  دیکھ  کر  تسلی  ہوئی  ہم  کو  اس  گھر  میں  جانتا  ہے  کوئی',
 'زندگی  یوں  ہوئی  بسر  تنہا  قافلہ  ساتھ  اور  سفر  تنہا',
 'آپ  کے  بعد  ہر  گھڑی  ہم  نے  آپ  کے  ساتھ  ہی  گزاری  ہے',
 'آئنہ  دیکھ  کر  تسلی  ہوئی  ہم  کو  اس  گھر  میں  جانتا  ہے  کوئی',
 'زندگی  یوں  ہوئی  بسر  تنہا  قافلہ  ساتھ  اور  سفر  تنہا',
 'شام  سے  آنکھ  میں  نمی  سی  ہے  آج  پھر  آپ  کی  کمی  سی  ہے']

# Basic Preprocessing

In [8]:
def remove_duplicates(poetry):
    unique_sentences = []
    seen = set()
    for sentence in poetry:
        if sentence not in seen:
            unique_sentences.append(sentence)
            seen.add(sentence)
    return unique_sentences

In [9]:
# basic_pp_sent = basic_preprocessing(poetryy)
no_dupes = remove_duplicates(poetry)

In [10]:
len(no_dupes)

1227

In [11]:
poetryy = []
for sentence in no_dupes:
    if sentence:
        sentence_list = [sentence]
    else:
        sentence_list = []
    poetryy.append(sentence_list)

In [12]:
len(poetryy)

1227

In [13]:
poetryy[:10]

[['آپ  کے  بعد  ہر  گھڑی  ہم  نے  آپ  کے  ساتھ  ہی  گزاری  ہے'],
 ['آئنہ  دیکھ  کر  تسلی  ہوئی  ہم  کو  اس  گھر  میں  جانتا  ہے  کوئی'],
 ['زندگی  یوں  ہوئی  بسر  تنہا  قافلہ  ساتھ  اور  سفر  تنہا'],
 ['شام  سے  آنکھ  میں  نمی  سی  ہے  آج  پھر  آپ  کی  کمی  سی  ہے'],
 ['کبھی  تو  چونک  کے  دیکھے  کوئی  ہماری  طرف  کسی  کی  آنکھ  میں  ہم  کو  بھی  انتظار  دکھے'],
 ['وقت  رہتا  نہیں  کہیں  ٹک  کر  عادت  اس  کی  بھی  آدمی  سی  ہے'],
 ['کتنی  لمبی  خاموشی  سے  گزرا  ہوں  ان  سے  کتنا  کچھ  کہنے  کی  کوشش  کی'],
 ['عادتاً  تم  نے  کر  دیے  وعدے  عادتاً  ہم  نے  اعتبار  کیا'],
 ['جس  کی  آنکھوں  میں  کٹی  تھیں  صدیاں  اس  نے  صدیوں  کی  جدائی  دی  ہے'],
 ['کوئی  خاموش  زخم  لگتی  ہے  زندگی  ایک  نظم  لگتی  ہے']]

## Tokenize the corpus in order to split it into a list of words

In [14]:
corpus = []
startingWords = []
endingWords = []
punctuation = string.punctuation

for poem in poetryy:
    for line in poem:
        lineToken = line.split()
        if(lineToken):
            startingWords.append(lineToken[0])
            endingWords.append(lineToken[-1])
        for word in lineToken:
            w = word.strip(punctuation)
            if (w == '’’' or w == '‘‘'):
                w = -1
            else:
                corpus.append(w)

In [15]:
print("Length of Corpus: ", len(corpus))
print(corpus[:10])

Length of Corpus:  21783
['آپ', 'کے', 'بعد', 'ہر', 'گھڑی', 'ہم', 'نے', 'آپ', 'کے', 'ساتھ']


## Generate n-gram models

### Unigram

In [16]:
def Unigram(corpus):
    freqDistribution = {}
    unigramProb = {}
    for token in corpus:
        if token in freqDistribution:
            freqDistribution[token] += 1
        else:
            freqDistribution[token] = 1 

    for i, f in freqDistribution.items():
        unigramProb[i] = f / len(corpus)
        
    return  freqDistribution, unigramProb

In [17]:
wordFreq, uniProb = Unigram(corpus)

In [18]:
# print("wordFreq",wordFreq)

In [19]:
# print("uniProb",uniProb[:10])

### Bigram

In [20]:
def Bigram(corpus, freq):
    bigrams = []
    bigramProb={}
    for i in range(0, len(corpus) - 1):
        bigrams.append((corpus[i], corpus[i+1]))
        
    freqDistribution=Counter(bigrams)

    for b, bf in freqDistribution.items():
        bigramProb[b]= bf / freq[(b[0])]
  
    return freqDistribution, bigramProb

In [21]:
BwordFreq, biProb = Bigram(corpus, wordFreq)

In [22]:
# print("BwordFreq",BwordFreq)

In [23]:
# print("biProb", biProb)

### Trigram

In [24]:
def Trigram(corpus, BwordFreq):
    trigrams = []
    trigramProb={}
    for i in range(0, len(corpus) - 2):
        trigrams.append((corpus[i], corpus[i+1], corpus[i+2]))
        
    freqDistribution=Counter(trigrams)

    for t, tf in freqDistribution.items():
        trigramProb[t]= tf / BwordFreq[ (t[0], t[1]) ]
  
    return  freqDistribution, trigramProb

In [25]:
TwordFreq, triProb= Trigram(corpus, BwordFreq)

In [26]:
# print("TwordFreq", TwordFreq)

In [27]:
# print("triProb", triProb[:10])

### Algo
    # For each of the stanzas
        # For each verse
            # Generate a random number in the range [7...10]
            # Select first word
            # Select subsequent words until end of verse
            #[bonus]If not the first verse, try to rhyme the last word with the last word of the previous verse
            # Print verse
    # Print empty line after stanza

### CFD -> next word with highest prob

In [28]:
def NextWordGenerator(clause, probs): 
    maxP= -999
    for i in probs: 
        if i[0] == clause:
            if probs[i] > maxP:
                maxP = probs[i]
                nextWord = i
    return nextWord, maxP

### Poem generation

In [29]:
def generate_verse(startingWords, endingWords, NextWordGenerator, triProb, previous_rhyme=None):
    verse = []
    verse_length = random.randint(7, 10)
    prompt = random.choice(startingWords)
    verse.append(prompt)
    
    for _ in range(verse_length - 1):  # -1 -> first word exists 
        nextWord = verse[-1]
        K, maxP = NextWordGenerator(nextWord, triProb)
        verse.append(K[1])
        
    # rhyming bonus
    if previous_rhyme is not None:
        rhyming_words = [word for word in endingWords if word.endswith(previous_rhyme[-1])]
        if rhyming_words:
            rhyme_word = random.choice(rhyming_words)
            verse[-1] = rhyme_word
    return verse

# Stores last word of each verse for rhyming in the next verse
def generate_poem(startingWords, endingWords, NextWordGenerator, triProb):
    num_stanzas = 4
    lines_per_stanza = 4
    previous_rhyme = None
    for stanza in range(num_stanzas):
        for verse_num in range(lines_per_stanza):
            verse = generate_verse(startingWords, endingWords, NextWordGenerator, triProb, previous_rhyme)
            print(' '.join(verse))
            previous_rhyme = verse[-1].split()[-1]
        print()  

In [30]:
noOflines = 4

#### Bigram vs Trigram performance

In [31]:
generate_poem(startingWords, endingWords, NextWordGenerator, biProb)

یہ کیا ہے کہ اس کے لیے آ کر
وہ بھی نہیں ہے کہ اس کے پر
غم کی طرح سے بھی نہیں ہے بھر
یہ کیا ہے کہ اس کے لیے کر

یہ کیا ہے کہ اس کے لیے بھر
شاخ سے بھی نہیں ہے کہ کر
جدائیوں کے لیے آ کر بھی نہیں ہے کہ کر
روز اک عمر بھر نہ ہو گیا ہوں پر

سارے زمانے میں بھی نہیں ہے کر
اپنی جگہ اس کے لیے آ کر کر
ہاں ٹھیک ہے کہ اس کے لیے آ کر
میں بھی نہیں ہے کہ اس کے لیے پر

اک عمر بھر نہ ہو گیا ہوں میں بھی اسٹار
ہم نے کیا ہے کہ اس کے لیے کر
بادباں ناز اہل نظر کو بھی نہیں ہے کہ کر
مائیں دروازوں پر نہیں ہے کہ اس بھر



In [32]:
generate_poem(startingWords, endingWords, NextWordGenerator, triProb)

دوست زمیں سے آنکھ رکھتے ہو سکتا جب
تھا آج پھر آپ حاصل بھی ادب
بکھیرتے رہو زباں کی آنکھ رکھتے ہو سکتا لب
ان تک کہ مرے قاتل نہ اے سبب

سلوٹیں ہیں سزائیں بھیج دی ہم احباب
دل اداس ہوتا یادوں کی آنکھ رکھتے ہو انقلاب
علاج یہ روز گئی بات چھیڑ دیتی ہے آئنہ احباب
سب خواہشیں پوری ہوں ان تک کہ سبب

اب اٹھتا نہیں کہیں ٹک کر تسلی احباب
طلسم تھا آج پھر آپ حاصل بھی انتظار ادب
وہ جا رہا وگرنہ زندگی یوں ہوئی ہم آنکھیں سبب
جدا ہوئے ہوتے تھے زمانے میرے زندگی ادب

دل اداس ہوتا یادوں کی آنکھ رکھتے انقلاب
میں جانتا ہی گزاری نہ اے احباب
وہی انداز ہے آئنہ دیکھ بھال کرے ادب
یہ روز گئی بات چھیڑ دیتی ہے انقلاب



In [33]:
# trigram wins here 

# Making sonnet rhyme

In [34]:
def generate_verse1(startingWords, endingWords, NextWordGenerator, triProb, previous_rhyme=None, rhyme_scheme=None):
    verse = []
    verse_length = random.randint(7, 10)  
    prompt = random.choice(startingWords)
    verse.append(prompt)
    for _ in range(verse_length - 1): 
        nextWord = verse[-1]
        K, maxP = NextWordGenerator(nextWord, triProb)
        verse.append(K[1])

    if previous_rhyme is not None:
        rhyming_words = [word for word in endingWords if word.endswith(previous_rhyme[-1])]
        if rhyming_words:
            rhyme_word = random.choice(rhyming_words)
            verse[-1] = rhyme_word
    return verse

# stanza rhymes 
def generate_stanza1(startingWords, endingWords, NextWordGenerator, triProb, rhyme_scheme):
    stanza = []
    for rhyme_type in rhyme_scheme:
        verse = generate_verse1(startingWords, endingWords, NextWordGenerator, triProb, previous_rhyme=stanza[-1] if stanza else None)
        stanza.append(verse)
    return stanza

def generate_poem1(startingWords, endingWords, NextWordGenerator, triProb, rhyme_scheme):
    num_stanzas = 4
    lines_per_stanza = len(rhyme_scheme)
    
    # sonnet -> line 1 & 3 AND line 2 & 4
    for _ in range(num_stanzas):
        for i in range(lines_per_stanza):
            if i % 2 == 0: 
                rhyme_type = 'A'
            else: 
                rhyme_type = 'B'
            
            verse = generate_verse1(startingWords, endingWords, NextWordGenerator, triProb, previous_rhyme=None, rhyme_scheme=rhyme_scheme)
            print(' '.join(verse))
            if i == lines_per_stanza - 1:
                print() 

#### Bigram vs Trigram performance

In [40]:
rhyme_scheme = ["A" , "B", "A", "B"]
generate_poem1(startingWords, endingWords, NextWordGenerator, biProb, rhyme_scheme)

مدتیں ہو گیا ہوں میں بھی نہیں
سو سو سو سو سو سو سو سو
تیرا نام سے بھی نہیں ہے کہ اس کے
خواہشیں پوری ہوں میں بھی نہیں ہے کہ

جس کے لیے آ کر بھی نہیں ہے کہ
تم نے کیا ہے کہ اس کے لیے
کتاب اور بھی نہیں ہے کہ اس
کچھ اور بھی نہیں ہے کہ اس کے

شعورؔ تم نے کیا ہے کہ اس کے
میں بھی نہیں ہے کہ اس کے لیے
مر جائیں ہم نے کیا ہے کہ اس
الگ بات نہیں ہے کہ اس کے

نہ ہو گیا ہوں میں بھی نہیں ہے کہ
اک عمر بھر نہ ہو گیا ہوں میں بھی نہیں
دل کو بھی نہیں ہے کہ اس
زمانہ بہت دیر تک کوئی ہم نے



In [41]:
rhyme_scheme = ["A" , "B", "A", "B"]
generate_poem1(startingWords, endingWords, NextWordGenerator, triProb, rhyme_scheme)

کبھی جو شرابوں میں جانتا ہی گزاری نہ
دعائیں یاد بے حساب آج پھر آپ حاصل بھی
کون آ نکلا ہے آئنہ دیکھ بھال کرے درد اے
اس زندگی یوں ہوئی ہم آنکھیں ابر تو

نئے حادثے پہ بھروسہ بھی انتظار دکھے وقت رہتا
اے کاش خوشبو جیسے لوگ ملے افسانے میں
بھول جاؤ سب خواہشیں پوری ہوں ان تک
کچھ ہم آنکھیں ابر تو چونک کے دیکھے کوئی

صرف ہاتھوں کو پگھلائیں تو چونک کے دیکھے کوئی
ایک نظم لگتی ہیں سزائیں بھیج دی ہم آنکھیں
نہ اے کاش خوشبو جیسے لوگ ملے افسانے میں جانتا
ڈھونڈتا پھرتا ہے آئنہ دیکھ بھال کرے درد اے کاش

یوں ہوئی ہم آنکھیں ابر تو چونک کے
کچھ ہم آنکھیں ابر تو چونک کے دیکھے کوئی
میں جانتا ہی گزاری نہ اے کاش خوشبو جیسے
پہلے پہلے پہلے پہلے پہلے پہلے پہلے پہلے



In [42]:
# !pip install pronouncing

In [43]:
# no 2 grams occurr together

In [44]:
def generate_verse2(startingWords, endingWords, NextWordGenerator, triProb, previous_rhyme=None, rhyme_scheme=None):
    verse = []
    verse_length = random.randint(7, 10) 
    prompt = random.choice(startingWords)
    verse.append(prompt)
    
    last_trigrams = [None, None]
    
    for _ in range(verse_length - 1):
        nextWord = verse[-1]
        
        # next word + geam 
        K, maxP = NextWordGenerator(nextWord, triProb)
        next_word = K[1]
        next_trigram = ' '.join([verse[-1]] + [next_word])
        
        attempts = 0
        max_attempts = 5  
        
        while next_trigram in last_trigrams and attempts < max_attempts:
            K, maxP = NextWordGenerator(nextWord, triProb)
            next_word = K[1]
            next_trigram = ' '.join([verse[-1]] + [next_word])
            attempts += 1
        
        # checks 5 times for unique gram, if its not found, orignal hi daldo
        if attempts == max_attempts:
            next_word = K[1] 
        
        # update the last two grams and remove the oldest
        verse.append(next_word)
        last_trigrams.append(next_trigram)
        last_trigrams.pop(0) 
        
    # Bonus
    if previous_rhyme is not None:
        rhyming_words = [word for word in endingWords if word.endswith(previous_rhyme[-1])]
        if rhyming_words:
            rhyme_word = random.choice(rhyming_words)
            verse[-1] = rhyme_word
            
    return verse

def generate_poem2(startingWords, endingWords, NextWordGenerator, triProb):
    num_stanzas = 4
    
    # Rhyme scheme of sonnet
    rhyme_scheme = ['ABAB'] * 3 + ['AAAA']
    
    for stanza_index in range(num_stanzas):
        lines_per_stanza = len(rhyme_scheme[stanza_index])
        
        for line_index in range(lines_per_stanza):
            rhyme_type = rhyme_scheme[stanza_index][line_index]
            
            verse = generate_verse2(startingWords, endingWords, NextWordGenerator, triProb, previous_rhyme=None, rhyme_scheme=rhyme_type)
            print(' '.join(verse))
        
        if stanza_index < num_stanzas - 1:
            print()

#### Bigram vs Trigram performance

In [49]:
generate_poem2(startingWords, endingWords, NextWordGenerator, biProb)

اس کے لیے آ کر بھی نہیں ہے کہ
اب تک کوئی ہم نے کیا ہے
دشت ہستی میں بھی نہیں ہے کہ اس کے لیے
جس کے لیے آ کر بھی نہیں ہے

آئین جواں مرداں حق ادا کہتی ہے کہ اس
یار نے کیا ہے کہ اس کے لیے آ کر
جس کے لیے آ کر بھی نہیں ہے
وہ بھی نہیں ہے کہ اس کے لیے آ کر

بند کلیوں کی طرح سے بھی نہیں ہے کہ
لوگ کہتے ہیں کہ اس کے لیے
کسی کو بھی نہیں ہے کہ اس
اس کے لیے آ کر بھی نہیں ہے

بہروپ نہیں ہے کہ اس کے لیے
کون سی بات نہیں ہے کہ اس
پھر بھی نہیں ہے کہ اس کے
مجھے تو نے کیا ہے کہ اس کے لیے


In [52]:
generate_poem2(startingWords, endingWords, NextWordGenerator, triProb)

عبرت آباد بھی انتظار دکھے وقت رہتا تم
ایک نظم لگتی ہیں سزائیں بھیج دی ہم آنکھیں ابر
سر بسر تنہا قافلہ ساتھ ہی گزاری نہ اے کاش
کڑا ہے آئنہ دیکھ بھال کرے درد

جانتا ہی گزاری نہ اے کاش خوشبو
مے پی ہے آئنہ دیکھ بھال کرے درد اے
تو چونک کے دیکھے کوئی خاموش زخم لگتی ہیں
بھری بہار پیلے پتے تلاش کرتی تھی

بے حساب آج پھر آپ حاصل بھی انتظار دکھے وقت
اس زندگی یوں ہوئی ہم آنکھیں ابر تو
بلاؤں گا وقت رہتا تم تکلف کو
ہر واقعہ تمہارا تھا آج پھر آپ

حق گوئی و لب یقیں آیا تب ماضی
رنگ حنا کا ہر واقعہ تمہارا تھا آج
ساقی تلخیٔ مے پی ہے آئنہ دیکھ بھال کرے درد
آج پھر آپ حاصل بھی انتظار دکھے


## Backward bigram model
model the generation of a sentence from right to left

In [53]:
def NextWordGenerator1(clause, probs): 
    maxP = -999
    for i in probs: 
        if i[0] == clause:
            if probs[i] > maxP:
                maxP = probs[i]
                nextWord = i
    return [nextWord[1]], [maxP]

In [54]:
def generate_verse3_BB(startingWords, endingWords, NextWordGenerator1, triProb, previous_rhyme=None):
    while True:
        verse = []
        verse_length = random.randint(7, 10) 
        prompt = random.choice(startingWords)
        verse.append(prompt)
        
        while len(verse) < verse_length:
            nextWord = verse[-1]
            nextWords, probabilities = NextWordGenerator1(nextWord, triProb)
            # nextWord is not repeated, if not available, brk 
            available_words = [word for word in nextWords if word not in verse]
            if available_words:
                nextWord = random.choices(available_words, weights=probabilities)[0]
            else:
                break
            verse.append(nextWord)
            
        # Bonus
        if previous_rhyme is not None:
            rhyming_words = [word for word in endingWords if word.endswith(previous_rhyme[-1])]
            if rhyming_words:
                rhyme_word = random.choice(rhyming_words)
                verse[-1] = rhyme_word
                
        if len(verse) >= 7 and len(verse) <= 10:
            break
    return verse

def generate_poem3_BB(startingWords, endingWords, NextWordGenerator1, triProb):
    num_stanzas = 4
    lines_per_stanza = 4
    previous_rhyme = None
    for stanza in range(num_stanzas):
        for verse_num in range(lines_per_stanza):
            verse = generate_verse3_BB(startingWords, endingWords, NextWordGenerator1, triProb, previous_rhyme)
            print(' '.join(verse))

            previous_rhyme = verse[-1].split()[-1]
        print() 

In [55]:
def BackwardBigram(corpus, freq, smoothing=1):
    backwardBigrams = []
    backwardBigramProb = {}
    
    # backward bigrams
    for i in range(1, len(corpus)):
        backwardBigrams.append((corpus[i], corpus[i-1]))
        
    # frequency count
    backwardBigramFreq = Counter(backwardBigrams)
    
    # Laplace Smoothing to avoid x/0
    for bb, bbf in backwardBigramFreq.items():
        word_freq = freq.get(bb[0], 0) 

        backwardBigramProb[bb] = (bbf + smoothing) / (word_freq + smoothing * len(freq))
    
    return backwardBigramFreq, backwardBigramProb

#### Bigram vs Trigram vs Backward Bigram performance

##### Bigram

In [56]:
BBwordFreq, BBiProb = BackwardBigram(corpus, BwordFreq)

In [61]:
generate_poem3_BB(startingWords, endingWords, NextWordGenerator1, BBiProb)

ہم ہیں جاتے بھول کو مجھ تو ہے کیا
کر دیکھ کو مجھ تو ہے رہنا
حق مرداں جواں آئین ہیں جاتے بھول کو ہوتا
عمل ہو نہ بھی میں ہوں میرا

ہم ہیں جاتے بھول کو مجھ تو ہے مارا
ہائے کرو نہ بھی میں ہوں ملا
کر دیکھ کو مجھ تو ہے ہمارا
شعورؔ گیا ہو نہ بھی میں ہوں آیا

مدتیں دوں کر دیکھ کو مجھ تو ہے کا
عشق ترے تھا نہ بھی میں ہوں آیا
مستقل ہو نہ بھی میں ہوں ہوگا
دلوں ہیں جاتے بھول کو مجھ تو ہوگا

عجب ہیں جاتے بھول کو مجھ تو ہے گا
ہم ہیں جاتے بھول کو مجھ تو ہے گا
چاندنی بھر عمر ہیں جاتے بھول کو کیا
آئین ہیں جاتے بھول کو مجھ تو ہے تھا



##### Trigram

In [62]:
BTwordFreq, BTiProb = BackwardBigram(corpus, TwordFreq)

In [69]:
generate_poem3_BB(startingWords, endingWords, NextWordGenerator1, BTiProb)

ترے تھا نہ بھی میں ہوں رہا
ہم ہیں جاتے بھول کو مجھ تو ہے گیا
فیضؔ مقام نہیں بھی میں ہوں جا
نشہ لو کر دیکھ کو مجھ تو ہونا

ہائے کرو نہ بھی میں ہوں آیا
جیسے ہو نہ بھی میں ہوں رہا
ستاروں دیکھ کو مجھ تو ہے ہوتا
اہل ہم ہیں جاتے بھول کو مجھ جاتا

بظاہر سکے نہ بھی میں ہوں گیا
مبہم سبب بے کو مجھ تو ہوگا
ضبط جیسے ہو نہ بھی میں ہوں تھا
بیٹیاں دے کر دیکھ کو مجھ تو ہوگا

ستاروں دیکھ کو مجھ تو ہے جاتا
موسم گا جائے ہو نہ بھی میں ہوں رکھا
اچھوں ہوئے ہوتے کے اس ہے دیکھا
یادوں ہوتا نہیں بھی میں ہوں میرا



In [70]:
# backward bigram wins

## Bidirectional bigram model
merge fwd and backward

In [71]:
def generate_verse4(startingWords, endingWords, NextWordGenerator, biProb, BBiProb, previous_rhyme=None):
    while True:
        verse = []
        verse_length = random.randint(7, 10) 
        prompt = random.choice(startingWords)
        verse.append(prompt)
        while len(verse) < verse_length:
            nextWord = verse[-1]
            # forward model
            K, maxP = NextWordGenerator(nextWord, biProb)
            # backward model
            K1, maxP1 = NextWordGenerator(nextWord, BBiProb)
            
            if maxP > maxP1:
                verse.append(K[1])
            else:
                verse.append(K1[1])
        
        if previous_rhyme is not None:
            rhyming_words = [word for word in endingWords if word.endswith(previous_rhyme[-1])]
            if rhyming_words:
                rhyme_word = random.choice(rhyming_words)
                verse[-1] = rhyme_word
        if len(verse) >= 7 and len(verse) <= 10:
            break
    return verse

def generate_poem4(startingWords, endingWords, NextWordGenerator, biProb, BBiProb):
    num_stanzas = 4
    lines_per_stanza = 4
    previous_rhyme = None
    for stanza in range(num_stanzas):
        for verse_num in range(lines_per_stanza):
            verse = generate_verse4(startingWords, endingWords, NextWordGenerator, biProb, BBiProb, previous_rhyme)
            print(' '.join(verse))
            previous_rhyme = verse[-1].split()[-1]
        print() 

#### Bigram vs Trigram vs Bidirectional Bigram performance

##### Bigram

In [80]:
generate_poem4(startingWords, endingWords, NextWordGenerator, biProb, BBiProb)

اک عمر بھر نہ ہو گیا ہوں میں بھی نہیں
دعا کے لیے آ کر بھی ہیں
انداز سے بھی نہیں ہے کہ اس کے تھیں
اک عمر بھر نہ ہو گیا گئیں

گھرا ہوا ہے کہ اس کے لیے آ نہیں
حد نظر کو بھی نہیں ہے کہ ہیں
اے دل کو بھی نہیں ہے کہ کریں
ہائے وہ بھی نہیں ہے کہ اس ہیں

کمان شاخ سے بھی نہیں ہے کہ جائیں
اس کے لیے آ کر بھی نہیں کہاں
زندگی بھر نہ ہو گیا ہوں میں بھی ہیں
تمام عمر بھر نہ ہو گیا ہوں ہوں

تم نے کیا ہے کہ اس میں
ہر بات نہیں ہے کہ اس لیں
اب تک کوئی ہم نے کیا ہے کہ ہیں
اب تک کوئی ہم نے کیا ہے ہوں



In [76]:
# bigram wins

##### Triigram

In [82]:
generate_poem4(startingWords, endingWords, NextWordGenerator, triProb, BTiProb)

نیندوں میں جانتا ہی گزاری نہ اے
چند امیدیں نچوڑی تھیں صدیاں اس زندگی ہے
خفا ہے آئنہ دیکھ بھال کرے درد اے کاش آئے
لو نشہ بڑھتا ہے آئنہ دیکھ بھال ہے

رہنا اکتائے ہوئے ہوتے تھے زمانے جائے
یا بدلاؤ سے آنکھ رکھتے ہو دیتے
اچھا نہ اے کاش خوشبو جیسے نے
بھوک سے آنکھ رکھتے ہو سکتا کہیے

مقصد زیست غم دنیا تھی ان تک ہے
تمام شہر والوں کو پگھلائیں تو چونک ہے
پلٹ سی خاموشی کا ہر واقعہ تمہارا تھا آج سے
ایک نظم لگتی ہیں سزائیں بھیج دی ہم آنکھیں ہوتے

اور دائرے ہیں سزائیں بھیج دی ہے
تکلیف کم نہیں کہیں ٹک کر تسلی ہوئی ہم کرتے
کیسے بولیں گے جدائی دی ہم آنکھیں ابر تو ہے
جی بھرا سا رہتا تم تکلف چاہئے

